In [ ]:
knitr::opts_chunk$set(echo = TRUE)


## Introduction

This document showcases a completely reproducible [particulate](https://en.wikipedia.org/wiki/Particulates) matter analysis.
Starting from the used measurement devices, via software for data hosting, to the analysis and visualisation environment.
It stands on the shoulders of communities who provide free and open resources in the spirit of [Open Science](https://en.wikipedia.org/wiki/Open_science):

- **open hardware**: the [senseBox](https://sensebox.de/en/) project
- **open data**: the [openSenseMap](https://opensensemap.org/) and it's [API](https://api.opensensemap.org/) provide environmental data licensed under [PDDL 1.0](http://opendatacommons.org/licenses/pddl/summary/)
- **free and open source software**: [software by the senseBox team](https://github.com/sensebox/) to host their services and [download the data into R](https://github.com/noerw/opensensmapR), [R](https://jupyter.org/) with a large number of packages (e.g. `rmarkdown`, `sf`, `dplyr`, ...), [Project Jupyter](https://jupyter.org/), [binder](https://mybinder.org/), [Rocker](https://www.rocker-project.org/), [Docker](https://docker.com/), ... and many more

The actual analysis is based on the [opensensmapR vignette `osem-intro`](https://noerw.github.io/opensensmapR/inst/doc/osem-intro.html).
The code and it's environment are published, documented, and packaged to support [reproducible research](https://doi.org/10.1045/january2017-nuest).

The code repository is [https://github.com/nuest/sensebox-binder](https://github.com/nuest/sensebox-binder) and the [git version hash](https://git-scm.com/docs/git-rev-parse) is `r system("git rev-parse HEAD", intern = TRUE)`.
The repository can be opened interactively at [http://mybinder.org/v2/gh/nuest/sensebox-binder/master](http://mybinder.org/v2/gh/nuest/sensebox-binder/master).
The code (this document as either [R Markdown](http://rmarkdown.rstudio.com/) or [Jupyter Notebook](https://nbformat.readthedocs.io/en/latest/)) and environment (a [Docker image](https://docs.docker.com/glossary/?term=image)) are archived with a [DOI](https://www.doi.org/): [`10.5281/zenodo.1135140`](https://doi.org/10.5281/zenodo.1135140).

**Note**: The data is not included in the archive so this document can only be compiled as long as the openSenseMap API exists.

## Analysis

In the remainder of this file, code "chunks" and text are [interspersed](https://en.wikipedia.org/wiki/Literate_programming) to provide a transparent and understandable workflow.

The analysis of takes a look at _fine particulate matter measured in Germany at New Year's Eve 2018_.

### Load required software



In [ ]:
library("opensensmapr")
library("dplyr")
library("lubridate")
library("units")
library("sf")
library("leaflet")


<span style="color: grey;">[output hidden]</span>

### Exploring openSenseMap



In [ ]:
all_boxes <- osem_boxes()
analysis_date <- lubridate::as_datetime("2018-01-01 00:00:00")
pm25_boxes <- osem_boxes(
  exposure = 'outdoor',
  date = analysis_date, # ±4 hours
  phenomenon = 'PM2.5',
  model = "homeWifiFeinstaub" # redundant because of phenomenon
)


The openSenseMap currently provides access to `r nrow(all_boxes)` senseBoxes of which `r nrow(pm25_boxes)` provide measurements of [PM2.5](https://www.umweltbundesamt.de/sites/default/files/medien/377/dokumente/infoblatt_feinstaub_pm2_5_en.pdf) around `r format(analysis_date, "%Y-%m-%d %T %Z")`.

The following map shows the PM2.5 sensor locations.



In [ ]:
plot(pm25_boxes)



### Particulates at New Year's Eve in Münster

_How many senseBoxes in Münster measure PM2.5 and what are the values?_



In [ ]:
ms <- st_sfc(st_point(c(7.62571, 51.96236)))
st_crs(ms) <- 4326

pm25_boxes_sf <- st_as_sf(pm25_boxes, remove = FALSE, agr = "identity")
names(pm25_boxes_sf) <- c(names(pm25_boxes), "geometry")

pm25_boxes_sf <- cbind(pm25_boxes_sf, dist_to_ms = st_distance(ms, pm25_boxes_sf))
max_dist <- set_units(3, km) # km from city center

ms_boxes <- pm25_boxes_sf[pm25_boxes_sf$dist_to_ms < max_dist,c("X_id", "name")]
ms_boxes


Now we retrieve data for `r nrow(ms_boxes)` senseBoxes with values in the area of interest.



In [ ]:
class(ms_boxes) <- c(class(ms_boxes), "sensebox")
ms_data <- osem_measurements(ms_boxes, phenomenon = "PM2.5",
                             from = lubridate::as_datetime("2017-12-31 20:00:00"),
                             to = lubridate::as_datetime("2018-01-01 04:00:00"))
summary(ms_data %>%
                select(value,createdAt,sensorId,unit))


We can now plot `r nrow(ms_data)` measurements.



In [ ]:
plot(value~createdAt, ms_data,
     type = "p", pch = '*', cex = 2, # new year's style
     col = factor(ms_data$sensorId),
     xlab = NA,
     ylab = unique(ms_data$unit),
     main = "Particulates measurements (PM2.5) on New Year 2017/2018",
     sub = paste(nrow(ms_boxes), "stations in Münster, Germany\n",
                 "Data by openSenseMap.org licensed under",
                 "Public Domain Dedication and License 1.0"))


You can see, it was a [very "particular" celebration](http://www.dw.com/en/new-years-eve-are-fireworks-harming-the-environment/a-41957523).

_Who is the record holder?_



In [ ]:
top_three <- ms_data %>%
  arrange(desc(value)) %>%
  head(n = 3)
knitr::kable(x = top_three,
             caption = "Top 3 measurements")


**Note:** The timestamp is UTC and the local time is [CET](https://en.wikipedia.org/wiki/CET) (`UTC+1:00`).



In [ ]:
# find the top station by its coordinates
top_sensor <- ms_data %>% arrange(desc(value)) %>% head(n = 1) %>% .$sensorId
top_box_coords <- ms_data %>% filter(sensorId == top_sensor) %>% select("lat", "lon") %>% slice(c(1))
top_box <- all_boxes %>% filter(lon == top_box_coords$lon, lat == top_box_coords$lat)

knitr::kable(data.frame(top_sensor, top_box$name),
             col.names = c("Top sensor identifier", "Top box name"))


Congratulations (?) to sensor _`r top_sensor`_ at box **`r top_box$name`** for holding the record values just after the new year started.

_Where is the record holding box?_

**Static plot**:



In [ ]:
plot(top_box)


**Interactive map** <span style="color: grey;">[Does not work in Jupyter Notebook]</span>:



In [ ]:
fireworks_icon <- makeIcon(
  # icon source: https://commons.wikimedia.org/wiki/File:Fireworks_2.png
  iconUrl = "320px-Fireworks_2.png",
  iconWidth = 160, iconHeight = 94)

leaflet(data = top_box) %>%
  addTiles() %>%
  addMarkers(~lon, ~lat, popup = ~as.character(name), label = ~as.character(name),
             icon = fireworks_icon)


## Jupyter Notebook conversion

A converted version of this file can in Jupyter Notebook format is automatically created with each rendering using [`ipyrmd`](https://pypi.python.org/pypi/ipyrmd/0.4.3).
`ipyrmd` is installed with other dependencies in the file `install.R`.
The Jupyter Notebook is intended to increase accessability for users unfamiliar with R Markdown.
The automatic conversion does not handle code statements within sentences.

```{bash}
ipyrmd --to ipynb --from Rmd -y -o sensebox-analysis.ipynb sensebox-analysis.Rmd
```

## Conclusion

This document creates a reproducible workflow of open data from a public API.
It leverages software to create a transparent analysis, which can be easily opened, investigated, and even developed further with a web browser by opening the public code repository on a free cloud platform.
To increase reproducibility, the data is cached manually as CSV files (i.e. text-based data format) and stored next to the analysis file.
A use may adjust this workflow to her own needs, like different location or time period, by adjust the R code and deleting the data files.
In case the exploration platform ceases to exist, users may still recreate the environment themselves based on the files in the code repository.
A snapshot of the files from the code repository, i.e. data, code, and runtime environment (as a Docker image) are stored in a reliable data repository.
While the manual workflow of building the image and running it is very likely to work in the future, the archived image captures the exact version of the software the original author used.

The presented solution might seem complex.
But it caters to many different levels of expertise (one-click open in browser vs. self-building of images and local inspection) and has several fail-safes (binder may disappear, GitHub repository may be lost, Docker may stop working).
The additional work is much outweighed by the advantages in transparency and openness.

## License

<img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" />

This document is licensed under a [Creative Commons Attribution 4.0 International ](https://creativecommons.org/licenses/by/4.0/) (CC BY 4.0).

## Metadata



In [ ]:
sessionInfo()